In [ ]:
import pickle
import os 
import sys
import torch
current_directory = os.getcwd()
parent_directory = os.path.dirname(os.path.dirname(current_directory))
sys.path.insert(0, parent_directory)
from data.dataset import PDBDataset
from models.classifier_model import ClassifierModel
from trainers.abs_trainer import Trainer

In [ ]:
dataset_path = "" # path to .pkl file of binding sites predicted by PeSTo
model_ckpt = "" # path to model ckpt, that is specific to a ligand

pion_model = torch.load(model_ckpt, map_location='cuda')
batch_size = 16

with open(dataset_path, "rb") as f:
    test_dataset = pickle.load(f)

predicted = []
for i in range(0, len(test_dataset), batch_size):
    end = min(i+batch_size, len(test_dataset))
    batch = PDBDataset.collate_fn([test_dataset[j]['data'] for j in range(i, end)])
    batch = Trainer.to_device(batch, 'cuda')
    output = pion_model.infer(batch).detach().cpu()
    predicted.append(output)
predicted = torch.cat(predicted, dim=0).squeeze()
